In [1]:
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import csv
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format



In [9]:
train_labels_12m = []
train_labels_1m = []
train_labels_3m = []
train_labels_6m = []
train_samples = []
data = open(r"C:\Users\cobyw\OneDrive\Documents\GitHub\ML-final-project\data\final_dataset_cleaned.csv")
reader = csv.reader(data)


for row in reader:
    if row[0] == "DATE":
        continue
    row = [eval(i) for i in row]
    train_samples.append(row[2:12])
    train_labels_1m.append(row[12])
    train_labels_3m.append(row[13])
    train_labels_6m.append(row[14])
    train_labels_12m.append(row[15])
train_labels_1m = np.asarray(train_labels_1m)
train_labels_3m = np.asarray(train_labels_3m)
train_labels_6m = np.asarray(train_labels_6m)
train_labels_12m = np.asarray(train_labels_12m)
train_samples = np.asarray(train_samples)
train_samples




array([[ 8.60000e+00,  5.41940e+03,  9.05650e+04, ...,  9.44000e+01,
        -1.91000e-02,  1.00000e+00],
       [ 8.90000e+00,  5.43940e+03,  9.05630e+04, ...,  9.47000e+01,
        -2.74000e-02,  1.00000e+00],
       [ 9.00000e+00,  5.43500e+03,  9.04340e+04, ...,  9.47000e+01,
         5.47000e-02,  1.00000e+00],
       ...,
       [ 3.80000e+00,  1.57109e+04,  1.56476e+05, ...,  3.06269e+02,
         1.31000e-02,  0.00000e+00],
       [ 3.80000e+00,  1.57415e+04,  1.56773e+05, ...,  3.07481e+02,
        -2.20000e-02,  0.00000e+00],
       [ 3.90000e+00,  1.57850e+04,  1.56923e+05, ...,  3.07619e+02,
         8.92000e-02,  0.00000e+00]])

In [10]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_train_samples = scaler.fit_transform(train_samples)

In [141]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
skf.get_n_splits(train_samples, train_labels_12m)
print(skf.split(train_samples, train_labels_12m))

<generator object _BaseKFold.split at 0x0000026470ACE810>


In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Conv1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy


In [12]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Availible: ", len(physical_devices))


Num GPUs Availible:  0


In [20]:
unit_array = [10,10,10]
mode = "relu"
input = [Dense(units=unit_array[0], input_shape=(10,), activation = mode)]

for i in range(1, len(unit_array)):
    input.append(Dense(units=unit_array[i], activation = mode))
input.append(Dense(units=2, activation="softmax"))
model = Sequential(input)
"""model = Sequential([
    Dense(units=unit_array[0], input_shape=(10,), activation = mode),
    , Dense(unit = unit_arry[i], activation = mode) for i in range(1, len(unit_array))
    Dense(units=2, activation="softmax")
])"""
#Dense(units=unit_array[1], activation = mode)

'model = Sequential([\n    Dense(units=unit_array[0], input_shape=(10,), activation = mode),\n    , Dense(unit = unit_arry[i], activation = mode) for i in range(1, len(unit_array))\n    Dense(units=2, activation="softmax")\n])'

In [21]:
keras.backend.clear_session()

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                110       
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 2)                 22        
                                                                 
Total params: 352 (1.38 KB)
Trainable params: 352 (1.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"],run_eagerly=True)

In [148]:
fitted_model = model.fit(x=scaled_train_samples, y=train_labels_12m, batch_size=3, epochs=5, shuffle=True, verbose=2)

Epoch 1/5
168/168 - 4s - loss: 0.8063 - accuracy: 0.0857 - 4s/epoch - 23ms/step
Epoch 2/5
168/168 - 4s - loss: 0.7065 - accuracy: 0.4283 - 4s/epoch - 21ms/step
Epoch 3/5
168/168 - 4s - loss: 0.5959 - accuracy: 0.9084 - 4s/epoch - 21ms/step
Epoch 4/5
168/168 - 3s - loss: 0.5135 - accuracy: 0.9143 - 3s/epoch - 20ms/step
Epoch 5/5
168/168 - 3s - loss: 0.4504 - accuracy: 0.9143 - 3s/epoch - 20ms/step


In [18]:
from sklearn.model_selection import StratifiedKFold, KFold, cross_val_score
from tensorflow.estimator import DNNClassifier
from scikeras.wrappers import KerasClassifier

In [29]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
estimator = KerasClassifier(build_fn=model, epochs=8, batch_size=3, verbose=0)
cv_scores = cross_val_score(estimator, train_samples, train_labels_12m,cv=skf,n_jobs=1)
print(cv_scores.mean())

INFO:tensorflow:Assets written to: C:\Users\cobyw\AppData\Local\Temp\tmpayv2itvk\assets


INFO:tensorflow:Assets written to: C:\Users\cobyw\AppData\Local\Temp\tmpayv2itvk\assets
c:\Users\cobyw\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


INFO:tensorflow:Assets written to: C:\Users\cobyw\AppData\Local\Temp\tmpchsh8d72\assets


INFO:tensorflow:Assets written to: C:\Users\cobyw\AppData\Local\Temp\tmpchsh8d72\assets
c:\Users\cobyw\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


INFO:tensorflow:Assets written to: C:\Users\cobyw\AppData\Local\Temp\tmp3p81hryd\assets


...

INFO:tensorflow:Assets written to: C:\Users\cobyw\AppData\Local\Temp\tmp4eyrctz6\assets


INFO:tensorflow:Assets written to: C:\Users\cobyw\AppData\Local\Temp\tmp4eyrctz6\assets
c:\Users\cobyw\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


INFO:tensorflow:Assets written to: C:\Users\cobyw\AppData\Local\Temp\tmp8u41x4sr\assets


INFO:tensorflow:Assets written to: C:\Users\cobyw\AppData\Local\Temp\tmp8u41x4sr\assets
c:\Users\cobyw\AppData\Local\Programs\Python\Python310\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


0.8565940594059406
